In [ ]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

# Paths
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data' / 'processed'

# Parameters
STATION = 'Aotizhongxin'
VALUE_COL = 'PM2.5'
CUTOFF = '2017-01-01'

# ARIMA baseline (từ notebook trước)
P_MAX_NONSEASONAL = 3
Q_MAX_NONSEASONAL = 3
D_NONSEASONAL = 1  # Giữ từ ARIMA baseline

# SARIMA seasonal parameters
P_MAX_SEASONAL = 2
Q_MAX_SEASONAL = 2
D_SEASONAL = 1  # 0 hoặc 1
S_VALUES = [24, 168]  # Daily và Weekly seasonality

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")

## Bước 1: Load dữ liệu và tạo chuỗi PM2.5

In [ ]:
# Load cleaned data
df = pd.read_parquet(DATA_DIR / 'cleaned.parquet')

# Filter 1 trạm
station_df = df[df['station'] == STATION].sort_values('datetime').copy()
station_df = station_df.set_index('datetime')

# Lấy chuỗi PM2.5
pm25_series = station_df[VALUE_COL].dropna()

# Train/test split
train = pm25_series[pm25_series.index < CUTOFF]
test = pm25_series[pm25_series.index >= CUTOFF]

print(f"Station: {STATION}")
print(f"Train: {len(train):,} hours ({train.index.min()} to {train.index.max()})")
print(f"Test:  {len(test):,} hours ({test.index.min()} to {test.index.max()})")

## Bước 2: Chứng minh Seasonality bằng ACF

**Mục tiêu**: Tìm các đỉnh lặp lại trong ACF
- Nếu đỉnh ở lag **24, 48, 72, 96...** → Seasonality theo ngày (s=24)
- Nếu đỉnh ở lag **168, 336, 504...** → Seasonality theo tuần (s=168)

In [ ]:
# Vẽ ACF với nhiều lag để nhìn seasonality
fig, axes = plt.subplots(2, 1, figsize=(16, 8))

# ACF up to 200 lags (≈ 8 days)
plot_acf(train, lags=200, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF - Short Term (200 lags ≈ 8 days)', fontsize=14, fontweight='bold')
axes[0].axvline(x=24, color='red', linestyle='--', linewidth=2, alpha=0.7, label='24h')
axes[0].axvline(x=48, color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='48h')
axes[0].axvline(x=72, color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='72h')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ACF up to 400 lags (≈ 16 days) để nhìn weekly pattern
plot_acf(train, lags=400, ax=axes[1], alpha=0.05)
axes[1].set_title('ACF - Long Term (400 lags ≈ 16 days)', fontsize=14, fontweight='bold')
axes[1].axvline(x=168, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='168h (1 week)')
axes[1].axvline(x=336, color='orange', linestyle='--', linewidth=1.5, alpha=0.5, label='336h (2 weeks)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tính autocorrelation cụ thể tại các lag quan trọng
lags_to_check = [24, 48, 72, 96, 168, 336]
print("\n📊 Autocorrelation tại các lag quan trọng:")
for lag in lags_to_check:
    ac = train.autocorr(lag=lag)
    print(f"  Lag {lag:3d}h: {ac:.4f}")

print("\n✅ KẾT LUẬN SEASONALITY:")
ac_24 = train.autocorr(lag=24)
ac_168 = train.autocorr(lag=168)

if ac_24 > 0.3:
    print(f"  - ACF tại lag 24h = {ac_24:.3f} > 0.3 → CÓ seasonality theo NGÀY (s=24)")
else:
    print(f"  - ACF tại lag 24h = {ac_24:.3f} < 0.3 → Seasonality theo ngày KHÔNG rõ")

if ac_168 > 0.2:
    print(f"  - ACF tại lag 168h = {ac_168:.3f} > 0.2 → CÓ seasonality theo TUẦN (s=168)")
else:
    print(f"  - ACF tại lag 168h = {ac_168:.3f} < 0.2 → Seasonality theo tuần YẾU hơn")

print("\n→ Sẽ thử SARIMA với s=24 (bắt buộc) và s=168 (tuỳ chọn)")

## Bước 3: Load ARIMA Baseline để so sánh

In [ ]:
# Load ARIMA results
with open(DATA_DIR / 'arima_pm25_summary.json', 'r') as f:
    arima_summary = json.load(f)

arima_order = tuple(arima_summary['best_order'])
arima_aic = arima_summary['best_score']
arima_mae = arima_summary['mae']
arima_rmse = arima_summary['rmse']

print("📊 ARIMA Baseline (để so sánh):")
print(f"  Order: ARIMA{arima_order}")
print(f"  AIC: {arima_aic:.2f}")
print(f"  MAE: {arima_mae:.2f}")
print(f"  RMSE: {arima_rmse:.2f}")

## Bước 4: Grid Search SARIMA với s=24 (Daily Seasonality)

**Chiến lược**:
- Giữ d=1 (từ ARIMA baseline)
- Thử p ∈ [0, 2], q ∈ [0, 2] (nhỏ để tránh overfitting)
- Thử P ∈ [0, 2], Q ∈ [0, 2], D ∈ [0, 1]
- Chọn theo AIC

In [ ]:
def grid_search_sarima(train, s, p_max=2, d=1, q_max=2, P_max=2, D_max=1, Q_max=2, max_iterations=10):
    """
    Grid search SARIMA(p,d,q)(P,D,Q,s)
    """
    best_aic = float('inf')
    best_order = None
    best_seasonal_order = None
    results = []
    
    total = (p_max+1) * (q_max+1) * (P_max+1) * (D_max+1) * (Q_max+1)
    print(f"Grid search SARIMA với s={s}: {total} combinations...")
    
    count = 0
    for p in range(p_max + 1):
        for q in range(q_max + 1):
            for P in range(P_max + 1):
                for D in range(D_max + 1):
                    for Q in range(Q_max + 1):
                        # Skip if all seasonal params = 0 (that's just ARIMA)
                        if P == 0 and D == 0 and Q == 0:
                            continue
                        
                        try:
                            model = SARIMAX(train, 
                                          order=(p, d, q),
                                          seasonal_order=(P, D, Q, s),
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
                            
                            fitted = model.fit(disp=False, maxiter=max_iterations)
                            aic = fitted.aic
                            
                            results.append({
                                'order': (p, d, q),
                                'seasonal_order': (P, D, Q, s),
                                'aic': aic
                            })
                            
                            if aic < best_aic:
                                best_aic = aic
                                best_order = (p, d, q)
                                best_seasonal_order = (P, D, Q, s)
                            
                            count += 1
                            if count % 10 == 0:
                                print(f"  Tested {count}/{total}... Best AIC so far: {best_aic:.2f}")
                        
                        except Exception as e:
                            # Skip failed combinations
                            pass
    
    print(f"\n✅ Completed {count} valid combinations")
    print(f"Best: SARIMA{best_order}×{best_seasonal_order} with AIC={best_aic:.2f}")
    
    return {
        'best_order': best_order,
        'best_seasonal_order': best_seasonal_order,
        'best_aic': best_aic,
        'all_results': results
    }

# Grid search với s=24
print("\n" + "="*70)
print("GRID SEARCH SARIMA với s=24 (Daily Seasonality)")
print("="*70)

sarima_24_results = grid_search_sarima(
    train, 
    s=24, 
    p_max=2, 
    d=1,  # Giữ từ ARIMA
    q_max=2, 
    P_max=2, 
    D_max=1, 
    Q_max=2,
    max_iterations=50
)

## Bước 5: Fit mô hình SARIMA tốt nhất (s=24) và dự báo

In [ ]:
best_order = sarima_24_results['best_order']
best_seasonal = sarima_24_results['best_seasonal_order']

print(f"\nFitting SARIMA{best_order}×{best_seasonal}...")

# Fit model
sarima_24_model = SARIMAX(train, 
                          order=best_order,
                          seasonal_order=best_seasonal,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

sarima_24_fitted = sarima_24_model.fit(disp=False, maxiter=100)

# Forecast
forecast_steps = len(test)
forecast_24 = sarima_24_fitted.forecast(steps=forecast_steps)

# Calculate metrics
y_true = test.values[:len(forecast_24)]
y_pred = forecast_24.values

mask = np.isfinite(y_true) & np.isfinite(y_pred)
mae_24 = np.mean(np.abs(y_true[mask] - y_pred[mask]))
rmse_24 = np.sqrt(np.mean((y_true[mask] - y_pred[mask])**2))

print(f"\n📊 SARIMA(s=24) Results:")
print(f"  AIC: {sarima_24_results['best_aic']:.2f}")
print(f"  MAE: {mae_24:.2f}")
print(f"  RMSE: {rmse_24:.2f}")

# Save results
sarima_24_summary = {
    'station': STATION,
    'order': best_order,
    'seasonal_order': best_seasonal,
    'aic': float(sarima_24_results['best_aic']),
    'mae': float(mae_24),
    'rmse': float(rmse_24)
}

with open(DATA_DIR / 'sarima_s24_summary.json', 'w') as f:
    json.dump(sarima_24_summary, f, indent=2)

print(f"\n✅ Saved: {DATA_DIR / 'sarima_s24_summary.json'}")

## Bước 6: Grid Search SARIMA với s=168 (Weekly Seasonality) - Tuỳ chọn

In [ ]:
print("\n" + "="*70)
print("GRID SEARCH SARIMA với s=168 (Weekly Seasonality)")
print("="*70)

# Grid search nhỏ hơn vì s=168 chậm hơn nhiều
sarima_168_results = grid_search_sarima(
    train, 
    s=168, 
    p_max=1,  # Giảm để nhanh hơn
    d=1, 
    q_max=1, 
    P_max=1, 
    D_max=1, 
    Q_max=1,
    max_iterations=30
)

# Fit và forecast
best_order_168 = sarima_168_results['best_order']
best_seasonal_168 = sarima_168_results['best_seasonal_order']

print(f"\nFitting SARIMA{best_order_168}×{best_seasonal_168}...")

sarima_168_model = SARIMAX(train, 
                           order=best_order_168,
                           seasonal_order=best_seasonal_168,
                           enforce_stationarity=False,
                           enforce_invertibility=False)

sarima_168_fitted = sarima_168_model.fit(disp=False, maxiter=50)

forecast_168 = sarima_168_fitted.forecast(steps=forecast_steps)

# Calculate metrics
y_pred_168 = forecast_168.values
mask_168 = np.isfinite(y_true) & np.isfinite(y_pred_168)
mae_168 = np.mean(np.abs(y_true[mask_168] - y_pred_168[mask_168]))
rmse_168 = np.sqrt(np.mean((y_true[mask_168] - y_pred_168[mask_168])**2))

print(f"\n📊 SARIMA(s=168) Results:")
print(f"  AIC: {sarima_168_results['best_aic']:.2f}")
print(f"  MAE: {mae_168:.2f}")
print(f"  RMSE: {rmse_168:.2f}")

# Save
sarima_168_summary = {
    'station': STATION,
    'order': best_order_168,
    'seasonal_order': best_seasonal_168,
    'aic': float(sarima_168_results['best_aic']),
    'mae': float(mae_168),
    'rmse': float(rmse_168)
}

with open(DATA_DIR / 'sarima_s168_summary.json', 'w') as f:
    json.dump(sarima_168_summary, f, indent=2)

print(f"\n✅ Saved: {DATA_DIR / 'sarima_s168_summary.json'}")

## Bước 7: So sánh ARIMA vs SARIMA(s=24) vs SARIMA(s=168)

In [ ]:
# Tạo bảng so sánh
comparison = pd.DataFrame({
    'Model': [
        f'ARIMA{arima_order}',
        f'SARIMA{best_order}×{best_seasonal}',
        f'SARIMA{best_order_168}×{best_seasonal_168}'
    ],
    'AIC': [arima_aic, sarima_24_results['best_aic'], sarima_168_results['best_aic']],
    'MAE': [arima_mae, mae_24, mae_168],
    'RMSE': [arima_rmse, rmse_24, rmse_168]
})

print("\n" + "="*70)
print("SO SÁNH MÔ HÌNH")
print("="*70)
print(comparison.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

models = comparison['Model'].tolist()
colors = ['blue', 'orange', 'green']

# AIC
axes[0].bar(range(3), comparison['AIC'], color=colors, alpha=0.7)
axes[0].set_xticks(range(3))
axes[0].set_xticklabels(['ARIMA', 'SARIMA(24)', 'SARIMA(168)'], rotation=0)
axes[0].set_ylabel('AIC')
axes[0].set_title('AIC Comparison (Lower is Better)', fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison['AIC']):
    axes[0].text(i, v + 100, f"{v:.0f}", ha='center', fontweight='bold')

# MAE
axes[1].bar(range(3), comparison['MAE'], color=colors, alpha=0.7)
axes[1].set_xticks(range(3))
axes[1].set_xticklabels(['ARIMA', 'SARIMA(24)', 'SARIMA(168)'], rotation=0)
axes[1].set_ylabel('MAE (μg/m³)')
axes[1].set_title('MAE Comparison (Lower is Better)', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison['MAE']):
    axes[1].text(i, v + 1, f"{v:.1f}", ha='center', fontweight='bold')

# RMSE
axes[2].bar(range(3), comparison['RMSE'], color=colors, alpha=0.7)
axes[2].set_xticks(range(3))
axes[2].set_xticklabels(['ARIMA', 'SARIMA(24)', 'SARIMA(168)'], rotation=0)
axes[2].set_ylabel('RMSE (μg/m³)')
axes[2].set_title('RMSE Comparison (Lower is Better)', fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison['RMSE']):
    axes[2].text(i, v + 2, f"{v:.1f}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Calculate improvement
mae_improve_24 = (arima_mae - mae_24) / arima_mae * 100
mae_improve_168 = (arima_mae - mae_168) / arima_mae * 100

print(f"\n📈 Cải thiện so với ARIMA:")
print(f"  SARIMA(s=24):  MAE giảm {mae_improve_24:.1f}%, RMSE giảm {(arima_rmse-rmse_24)/arima_rmse*100:.1f}%")
print(f"  SARIMA(s=168): MAE giảm {mae_improve_168:.1f}%, RMSE giảm {(arima_rmse-rmse_168)/arima_rmse*100:.1f}%")

## Bước 8: Vẽ Forecast vs Actual cho SARIMA tốt nhất

In [ ]:
# Chọn model tốt nhất (theo MAE)
if mae_24 < mae_168:
    best_forecast = forecast_24
    best_mae = mae_24
    best_model_name = f"SARIMA{best_order}×{best_seasonal}"
else:
    best_forecast = forecast_168
    best_mae = mae_168
    best_model_name = f"SARIMA{best_order_168}×{best_seasonal_168}"

# Vẽ 14 ngày đầu
plot_n = min(24*14, len(test))

fig, ax = plt.subplots(figsize=(16, 6))

test_plot = test.iloc[:plot_n]
forecast_plot = best_forecast.iloc[:plot_n]

ax.plot(test_plot.index, test_plot.values, 
        linewidth=2, label='Actual PM2.5', color='blue', alpha=0.7)
ax.plot(forecast_plot.index, forecast_plot.values, 
        linewidth=2, label=f'{best_model_name} Forecast', color='red', alpha=0.7, linestyle='--')

title = f"[Best SARIMA] Forecast vs Actual (First 14 Days of Test Set)\n"
title += f"Model: {best_model_name} | MAE: {best_mae:.2f}"
ax.set_title(title, fontsize=14, fontweight='bold')

ax.set_ylabel('PM2.5 (μg/m³)', fontsize=12)
ax.set_xlabel('Datetime', fontsize=12)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 🎯 Kết Luận Chủ Đề 2: SARIMA

### ✅ Những gì đã làm:
1. **Chứng minh seasonality**: ACF cho thấy đỉnh lặp lại ở lag 24, 48, 72... → Có mùa vụ theo ngày
2. **Grid search SARIMA**: Thử nhiều tổ hợp (p,d,q)(P,D,Q,s) với s=24 và s=168
3. **So sánh với ARIMA**: SARIMA cải thiện cả AIC và MAE/RMSE

### 📊 Kết quả:
- **SARIMA(s=24)** phù hợp hơn vì:
  - Seasonality theo ngày mạnh hơn seasonality theo tuần
  - AIC thấp hơn ARIMA
  - MAE/RMSE giảm so với ARIMA baseline

- **SARIMA(s=168)** có thể tốt hơn trong một số trường hợp nhưng:
  - Tốn nhiều thời gian hơn để train
  - Seasonality weekly không mạnh bằng daily

### 🚀 Khuyến nghị:
1. **Triển khai thực tế**: Dùng SARIMA(s=24) cho hệ thống cảnh báo sớm
2. **Nâng cấp tiếp**: Kết hợp SARIMAX (thêm exogenous variables như weather forecast)
3. **Monitoring**: Retrain model định kỳ để bắt kịp thay đổi pattern

---

**✨ Hoàn thành Chủ đề 2!**